# <span style="color:red">QPM: Assignment 5</span>

###  LAFTIT Mehdi, LIN Christine, MUSEUX Célia and YANG Hexuan 

Financial Engineering - Quantitative Portfolio Management

**Due date :** 17/11/2023 9am

**Ressource:** Tidy finance website / Danny Groves


## <span style="color:green">Preliminary step</span>

We import the libraries we are going to use in this assignment:

In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate
import scipy.stats as stats
import scipy.optimize as opt
from scipy.optimize import minimize

Just like for Assignment 4, our dataset consists of monthly returns for these 10 companies. We continue to assume that the risk-free rate of return is zero.

We import our monthly log returns:

In [56]:
log_return_matrix = pd.read_csv("Monthly_Log_returns_10_companies.csv")
log_return_matrix.head(10)

,A,ABT,ADBE,ADM,ADP,AES,AFL,AKAM,AOS,MMM
0,0.450696,0.013346,0.616504,-0.150862,-0.085217,0.044995,-0.170378,0.047523,-0.135341,-0.053376
1,0.001203,0.064184,0.087614,0.024541,0.104225,-0.062304,0.220062,-0.485239,0.042560,0.004243
2,-0.159977,0.093346,0.082932,-0.037042,0.109110,0.132837,0.068901,-0.486383,0.139391,-0.022120
3,-0.185430,0.056888,-0.071765,0.188230,0.021827,-0.030337,0.058973,-0.392902,0.021053,-0.003692
4,0.001696,0.090972,0.144227,-0.196028,-0.024957,0.044825,-0.117934,0.575935,-0.003727,-0.032595
5,-0.593225,-0.066771,-0.126960,-0.045610,-0.077615,0.158057,0.122759,-0.409223,-0.252484,0.081663
6,0.403418,0.053155,0.126960,-0.006205,0.185103,0.174739,0.040679,-0.042706,-0.023531,0.038304
7,-0.220330,0.083196,0.177671,-0.023278,0.115892,0.073810,0.179328,-0.363849,-0.226124,-0.020535
8,-0.055132,0.108583,-0.020334,0.243231,-0.023642,-0.192821,0.122714,-0.029285,0.126862,0.058605
9,0.119431,0.041721,-0.182486,0.152218,0.010472,-0.085404,-0.036265,-0.573188,0.162961,0.039230


## <span style="color:green">Question 1 of Assignment 5</span>

In this assignment our universe selection is: 10 US publicly traded stocks (A, ABT, ADBE, ADM, ADP, AES, AFL, AOS, AKAM and MMM)


We set the estimation window to be T<sup>est</sup> = 60 months of monthly returns, and we will use this estimation sample to compute the 2 following portfolio strategies (mean-variance portfolio MVP and global minimum variance portfolio GMV)

We assume this time that there are nonnegativity constraints on the weights.

In [57]:
# We choose the estimation window T_est = 60 months
T_est = 60

In [58]:
# We set our estimation window
estimation_sample = log_return_matrix.iloc[:T_est, :]

print(estimation_sample.shape)
estimation_sample.head(10)

(60, 10)


,A,ABT,ADBE,ADM,ADP,AES,AFL,AKAM,AOS,MMM
0,0.450696,0.013346,0.616504,-0.150862,-0.085217,0.044995,-0.170378,0.047523,-0.135341,-0.053376
1,0.001203,0.064184,0.087614,0.024541,0.104225,-0.062304,0.220062,-0.485239,0.042560,0.004243
2,-0.159977,0.093346,0.082932,-0.037042,0.109110,0.132837,0.068901,-0.486383,0.139391,-0.022120
3,-0.185430,0.056888,-0.071765,0.188230,0.021827,-0.030337,0.058973,-0.392902,0.021053,-0.003692
4,0.001696,0.090972,0.144227,-0.196028,-0.024957,0.044825,-0.117934,0.575935,-0.003727,-0.032595
5,-0.593225,-0.066771,-0.126960,-0.045610,-0.077615,0.158057,0.122759,-0.409223,-0.252484,0.081663
6,0.403418,0.053155,0.126960,-0.006205,0.185103,0.174739,0.040679,-0.042706,-0.023531,0.038304
7,-0.220330,0.083196,0.177671,-0.023278,0.115892,0.073810,0.179328,-0.363849,-0.226124,-0.020535
8,-0.055132,0.108583,-0.020334,0.243231,-0.023642,-0.192821,0.122714,-0.029285,0.126862,0.058605
9,0.119431,0.041721,-0.182486,0.152218,0.010472,-0.085404,-0.036265,-0.573188,0.162961,0.039230


We compute the mu and sigma that we will be used in the calculation of the weights

In [59]:
mu = estimation_sample.mean()
sigma = estimation_sample.cov()

In [60]:
print("\033[1mThe means returns:\033[0m ")
mu

The means returns: 


A      -0.018274
ABT     0.008258
ADBE    0.012215
ADM     0.015106
ADP    -0.000593
AES    -0.017464
AFL     0.010640
AKAM   -0.049089
AOS     0.007296
MMM     0.011540
dtype: float64

In [61]:
print("\033[1mThe variance-covariance matrix:\033[0m ")
sigma

The variance-covariance matrix: 


,A,ABT,ADBE,ADM,ADP,AES,AFL,AKAM,AOS,MMM
A,0.034629,0.000556,0.018061,0.001473,0.005276,0.011865,-0.001762,0.032847,0.004939,0.001407
ABT,0.000556,0.005023,0.001637,0.001012,0.001420,-0.001031,0.000653,0.003172,0.000231,0.000153
ADBE,0.018061,0.001637,0.026524,-0.002333,0.003615,0.011494,0.001172,0.017048,-0.000584,0.001303
ADM,0.001473,0.001012,-0.002333,0.007340,0.002644,0.002321,0.001805,0.001677,0.001144,0.001749
ADP,0.005276,0.001420,0.003615,0.002644,0.006126,0.006776,0.001537,0.006731,0.000302,0.001088
AES,0.011865,-0.001031,0.011494,0.002321,0.006776,0.074649,0.003096,0.032154,0.005151,-0.000267
AFL,-0.001762,0.000653,0.001172,0.001805,0.001537,0.003096,0.006155,-0.002042,0.000833,0.001695
AKAM,0.032847,0.003172,0.017048,0.001677,0.006731,0.032154,-0.002042,0.108487,0.002554,0.002139
AOS,0.004939,0.000231,-0.000584,0.001144,0.000302,0.005151,0.000833,0.002554,0.012739,0.000490
MMM,0.001407,0.000153,0.001303,0.001749,0.001088,-0.000267,0.001695,0.002139,0.000490,0.003755


In [62]:
N = mu.shape[0]
print("\033[1mN =\033[0m ", N)
ones = np.ones(N) #vector of 1

N =  10


### 1.a Mean-variance portfolio (MVP) with constraints on the size of the weights and short-sale constraints

We define the objective function to be minimized

In [63]:
Rf=0

In [64]:
def objective1(weights):
    portfolio_return = weights.T @ (mu - Rf*ones)
    portfolio_risk = weights.T @ sigma @ weights
    
    return (gamma/2) * portfolio_risk - portfolio_return

Gamma represent the risk aversion (we have chosen it arbitrarly):

In [65]:
gamma=2

In [66]:
# We define the initial guess for weights
initial_weights = np.ones(len(mu))/len(mu)

#We define the equality constraint (weights sum to 1)
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) -1})

# We add a lower-bound constraint of 0 to the above problem. Since there is no upper bound, we have set it to 999
bounds = tuple((0, 999) for _ in range(len(mu)))

result = minimize(objective1, initial_weights, method='SLSQP', bounds = bounds, constraints=constraints)
optimal_weights1 = result.x

#Weights in the risk-free asset
w_riskFree = 0
w_mvp_shortsaleConstrained = list(map(lambda num: format(num, '.6f'), optimal_weights1))

In [67]:
print("\033[1mUsing numerical solver and with short-sale constraints\033[0m")
print("\033[1m\nWeights of the mean-variance portfolio (MVP-C) are =\033[0m ")
for i, weight in enumerate(optimal_weights1):
    print(f"Stock {i + 1}: {weight:.6f}")

Using numerical solver and with short-sale constraints

Weights of the mean-variance portfolio (MVP-C) are = 
Stock 1: 0.000000
Stock 2: 0.000000
Stock 3: 0.128858
Stock 4: 0.504111
Stock 5: 0.000000
Stock 6: 0.000000
Stock 7: 0.067329
Stock 8: 0.000000
Stock 9: 0.000000
Stock 10: 0.299701


In [68]:
optimal_weights1_sum=np.sum(optimal_weights1)
optimal_weights1_sum

1.0

We have verified that the sum of the weights is equal to 1.

### 1.b Global minimum variance (GMV)  with constraints on the size of the weight and short-sale constraints

We define the objective function to be minimized (portfolio variance)

In [69]:
def objective2(weights, sigma):
    portfolio_variance = weights.T @ sigma @ weights
    
    return portfolio_variance

In [71]:
#We define the equality constraint (weights sum to 1)
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) -1})


#We define the initial guess for weights
initial_weights = np.ones(len(sigma)) / len(sigma)

# Define the bounds for each weight (0 <= weight <= 999)
bounds = tuple((0, 999) for _ in range(len(sigma)))


result = minimize(objective2, initial_weights, args=(sigma,), method='SLSQP', bounds=bounds, constraints=constraints)
optimal_weights2 = result.x

In [74]:
print("\033[1mUsing numeric solution with short-sale constraints\033[0m")
print("\033[1m\nWeights of the Global Minimum Variance portfolio (GMV-C) are =\033[0m")
for i, weight in enumerate(optimal_weights2):
    print(f"Stock {i + 1}: {weight:.6f}")
print("\033[1m\nMinimum Portfolio Variance is:\033[0m", result.fun)

Using numeric solution with short-sale constraints

Weights of the Global Minimum Variance portfolio (GMV-C) are =
Stock 1: 0.000000
Stock 2: 0.294329
Stock 3: 0.025332
Stock 4: 0.064538
Stock 5: 0.084255
Stock 6: 0.003484
Stock 7: 0.138793
Stock 8: 0.000000
Stock 9: 0.091314
Stock 10: 0.297956

Minimum Portfolio Variance is: 0.0017798423124595385


In [75]:
optimal_weights2_sum=np.sum(optimal_weights2)
optimal_weights2_sum

1.0000000000000002

We have checked that the sum of the weights is 1. The variance is very small (0.18%)

## <span style="color:green">Question 2 of Assignment 5</span>

Now we use a rolling window of T_est = 60 months to estimate the portfolio weights for the two strategies listed above for each of the T − T_est months. That is, we repeat the calculations of the previous question for all the dates after the first 60 months.

In [76]:
#We specify the estimation window size
T_est = 60

#We get the number of assets
N = log_return_matrix.shape[1]

#We keep the same risk aversion parameter
gamma = 2

We can now compute the rolling weights:

In [77]:
#We initialize lists to store results
rolling_weights1 = []
rolling_weights2 = []
rolling_min_variance = []

In [78]:
#We iterate through the data with a rolling window
for i in range(T_est, len(log_return_matrix)):
    # Set the estimation window
    estimation_sample = log_return_matrix.iloc[i - T_est:i, :]

    #We calculate mean returns and covariance matrix
    mu = np.array(estimation_sample.mean()).T
    sigma = np.array(estimation_sample.cov())
    sigma_inv = np.linalg.inv(sigma)

    #We define the objective functions
    def objective1(weights):
        portfolio_return = weights.T @ mu
        portfolio_risk = weights.T @ sigma @ weights
        return (gamma / 2) * portfolio_risk - portfolio_return

    def objective2(weights):
        portfolio_variance = weights.T @ sigma @ weights
        return portfolio_variance

    #We define the initial guess for weights
    initial_weights = np.ones(N) / N

    #We define the equality constraint (weights sum to 1)
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

    #We define the bounds for each weight (0 <= weight <= 999)
    bounds = tuple((0, 999) for _ in range(N))

    #We perform mean-variance optimization with short-sale constraints
    result1 = minimize(objective1, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    optimal_weights1 = result1.x

    #We perform minimum variance optimization with short-sale constraints
    result2 = minimize(objective2, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    optimal_weights2 = result2.x

    #Wz store the results
    rolling_weights1.append(optimal_weights1)
    rolling_weights2.append(optimal_weights2)
    rolling_min_variance.append(result2.fun)

In [79]:
#We convert the lists to NumPy arrays for easier manipulation
rolling_weights1 = np.array(rolling_weights1)
rolling_weights2 = np.array(rolling_weights2)


#We convert the lists to pandas DataFrames
columns = [f"Stock {i + 1}" for i in range(N)]
index = [f"Window {i + 1}" for i in range(len(rolling_weights1))]
df_weights1 = pd.DataFrame(rolling_weights1, columns=columns, index=index)
df_weights2 = pd.DataFrame(rolling_weights2, columns=columns, index=index)

We display the results:

In [80]:
print("\033[1m\nMVP-C Rolling Weights:\033[0m")
df_weights1


MVP-C Rolling Weights:


,Stock 1,Stock 2,Stock 3,Stock 4,Stock 5,Stock 6,Stock 7,Stock 8,Stock 9,Stock 10
Window 1,0.000000e+00,5.025581e-18,1.288581e-01,0.504111,0.000000e+00,0.000000e+00,6.732943e-02,3.557897e-17,3.584642e-17,2.997010e-01
Window 2,8.706798e-17,0.000000e+00,0.000000e+00,0.574838,1.634828e-17,7.338519e-17,2.017880e-01,0.000000e+00,0.000000e+00,2.233742e-01
Window 3,0.000000e+00,3.330007e-17,2.663574e-17,0.620973,6.552512e-17,9.823607e-18,0.000000e+00,0.000000e+00,3.364657e-03,3.756621e-01
Window 4,1.645703e-17,3.664577e-02,3.604197e-17,0.297007,3.124391e-17,3.571457e-17,1.878890e-01,0.000000e+00,3.749291e-02,4.409658e-01
Window 5,8.777084e-18,0.000000e+00,4.387394e-17,0.175055,0.000000e+00,4.194963e-17,1.335939e-01,2.671378e-17,9.566853e-02,5.956825e-01
...,...,...,...,...,...,...,...,...,...,...
Window 211,0.000000e+00,3.602763e-01,3.919490e-01,0.160756,7.745181e-02,9.566813e-03,5.117434e-17,0.000000e+00,2.007942e-16,0.000000e+00
Window 212,9.010009e-17,1.176777e-01,3.244629e-02,0.288500,3.726824e-01,1.886941e-01,1.166927e-16,0.000000e+00,7.203352e-17,6.452010e-17
Window 213,4.163336e-17,1.856643e-01,0.000000e+00,0.251573,3.362803e-01,2.264823e-01,0.000000e+00,0.000000e+00,3.642919e-17,1.769418e-16
Window 214,0.000000e+00,0.000000e+00,0.000000e+00,0.515421,1.725862e-01,3.119924e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [81]:
print("\033[1m\nGMV-C Rolling Weights:\033[0m")
df_weights2


GMV-C Rolling Weights:


,Stock 1,Stock 2,Stock 3,Stock 4,Stock 5,Stock 6,Stock 7,Stock 8,Stock 9,Stock 10
Window 1,0.000000e+00,0.294329,2.533193e-02,0.064538,0.084255,3.483600e-03,0.138793,4.172308e-19,9.131368e-02,2.979564e-01
Window 2,0.000000e+00,0.271784,0.000000e+00,0.073023,0.104062,7.390459e-04,0.168569,7.635097e-19,9.340744e-02,2.884160e-01
Window 3,0.000000e+00,0.260839,2.313886e-17,0.069145,0.114077,0.000000e+00,0.206830,0.000000e+00,9.194295e-02,2.571658e-01
Window 4,0.000000e+00,0.274158,5.896181e-18,0.032223,0.131340,7.517256e-19,0.229730,0.000000e+00,1.108836e-01,2.216663e-01
Window 5,0.000000e+00,0.274094,0.000000e+00,0.045729,0.131510,6.665037e-19,0.228758,0.000000e+00,1.057098e-01,2.141990e-01
...,...,...,...,...,...,...,...,...,...,...
Window 211,9.492697e-04,0.235895,0.000000e+00,0.172225,0.158255,3.184442e-02,0.105491,2.691687e-01,1.734723e-18,2.617184e-02
Window 212,2.460684e-02,0.269706,1.474515e-17,0.177069,0.187484,1.751139e-02,0.083618,2.400036e-01,6.071532e-18,0.000000e+00
Window 213,8.846378e-03,0.257425,0.000000e+00,0.170995,0.174912,1.318595e-02,0.116995,2.576422e-01,0.000000e+00,1.040834e-17
Window 214,1.063219e-02,0.285552,5.149960e-18,0.133261,0.198227,6.129824e-03,0.107267,2.589309e-01,2.168404e-18,1.387779e-17


## <span style="color:green">Question 3 of Assignment 5</span>

We now use the time-series of portfolios weights for each of the two portfolio strategies to compute the out-of-sample portfolio returns. That is, for each of the two portfolio strategies that we estimate at each date $t$, we compute its out-of-sample return in month $t + 1$.

We define a function to compute portfolio returns

In [82]:
def portfolio_returns(weights, returns):
    portfolio_returns = np.dot(returns, weights)
    return portfolio_returns

We use the function `portfolio_returns` to compute the out-of-sample returns

In [83]:
mvp_returns_list = []
gmv_returns_list = []

# We calculate out-of-sample returns using the time-series of weights
for i in range(T_est, len(log_return_matrix)):
    current_returns = log_return_matrix.iloc[i, :]

    # MVP portfolio returns
    mvp_returns = portfolio_returns(current_returns, df_weights1.iloc[i - T_est])
    mvp_returns_list.append(mvp_returns)

    # GMV portfolio returns
    gmv_returns = portfolio_returns(current_returns, df_weights2.iloc[i - T_est])
    gmv_returns_list.append(gmv_returns)

# We convert the lists to NumPy arrays
MVP_C_returns = np.array(mvp_returns_list)
GMV_C_returns = np.array(gmv_returns_list)

We display our results:

In [84]:
results = pd.DataFrame({
    'MVP-C Returns': MVP_C_returns,
    'GMV-C Returns': GMV_C_returns,
})

table = tabulate(results, headers='keys', tablefmt='pretty', showindex=True)
print(table)

+-----+------------------------+-------------------------+
|     |     MVP-C Returns      |      GMV-C Returns      |
+-----+------------------------+-------------------------+
|  0  |  0.008348594855991312  |  0.0012421597058810572  |
|  1  |  0.010232535526346908  |  0.020165655202607453   |
|  2  |  -0.23660528833923106  |  -0.02207159176766696   |
|  3  |  0.04148461694297102   |  0.017208796902482792   |
|  4  |  -0.03170964395462003  |  -0.017558415015914545  |
|  5  |  0.05706236078108874   |   0.02020454430519079   |
|  6  | -0.013449672426085144  |  -0.028466102831796243  |
|  7  |  0.07272938834786194   |  0.007755724375836218   |
|  8  |  0.03420599261192278   |   0.04924185635126602   |
|  9  |  0.029493115745007664  |  -0.021169947173566447  |
| 10  |  0.006381786369923488  |  0.0016353201242040455  |
| 11  |  0.22693063059274393   |  0.062465225873614263   |
| 12  |  0.03611543671159041   |  0.017559739798317636   |
| 13  |   0.0634868825011084   | -1.5901655675561177e-05

## <span style="color:green">Question 4 of Assignment 5</span>

We compute the Sharp ratio of the out-of-sample returns, knowing that the risk-free rate of return is zero. We annualize the Sharpe Ratio

In [85]:
Rf = 0

MVP_sharpe_ratio = ((MVP_C_returns.mean() - Rf) / MVP_C_returns.std())
GMV_sharpe_ratio = ((GMV_C_returns.mean() - Rf) / GMV_C_returns.std())

print("The MVP Sharpe Ratio is:", MVP_sharpe_ratio)
print("The GMV Sharpe Ratio is:", GMV_sharpe_ratio)

The MVP Sharpe Ratio is: 0.12096898105357602
The GMV Sharpe Ratio is: 0.20335879589731362


In [86]:
ann_MVP_sharpe_ratio = MVP_sharpe_ratio* np.sqrt(12)
ann_GMV_sharpe_ratio = GMV_sharpe_ratio* np.sqrt(12)

print("The annualized MVP Sharpe Ratio is:", ann_MVP_sharpe_ratio)
print("The annualized GMV Sharpe Ratio is:", ann_GMV_sharpe_ratio)

The annualized MVP Sharpe Ratio is: 0.4190488426492611
The annualized GMV Sharpe Ratio is: 0.704455533320353


**Interpretation**: The GMV portfolio still have a higher Sharpe ratio than the MVP. However, we can see that, with a the short-sale constraints, the MVP Sharpe Ratio is higher than without constraints on weights.

## <span style="color:green">Final step</span>

We are finally asked to save the data we have downloaded for a future use.

In [87]:
file_path = "Monthly_Log_returns_10_companies.csv"
log_return_matrix.to_csv(file_path, index=False)

print(f"Data saved to {file_path}")

Data saved to Monthly_Log_returns_10_companies.csv
